# Connecting and Uploading Data to TigerGraph
This notebook will demonstrate connecting to an existing TigerGraph database instance, publishing a graph schema, and uploading data to our graph. To get started, create and launch a free cluster on [TigerGraph Cloud](tgcloud.io) and publish an empty graph in GraphStudio named 'Ethereum'.

In [41]:
# Imports
import pyTigerGraph as tg
import json
import os

# Import TigerGraph instance config
os.chdir('../config/')
with open('tigergraph.json', 'r') as f:
    config = json.load(f)

# Connection parameters
hostName = config['host']
secret = config['secret']

conn = tg.TigerGraphConnection(host=hostName, gsqlSecret=secret, graphname="Ethereum")
conn.getToken(secret)

('7jn0cs0ha36du2t16804r9jbkpe8pmgt', 1677976411, '2023-03-05 00:33:31')

### Define and Publish Graph Schema
In this section, we will publish our graph schema to TigerGraph, from which we will construct our Ethereum transaction graph. The graph schema is as follows:
* Nodes = Wallets <br />
* Directed Edges = Transactions

In [35]:
# DEFINE / CREATE ALL EDGES AND VERTICES in Global View
results = conn.gsql('''
  USE GRAPH Ethereum
  CREATE SCHEMA_CHANGE JOB build_schema FOR GRAPH Ethereum {
      ADD VERTEX Wallet (PRIMARY_ID id INT, label FLOAT, has_label BOOL, is_train BOOL, is_test BOOL, in_degree INT, out_degree INT, send_min DOUBLE, total_sent DOUBLE, recv_min DOUBLE, recv_amount DOUBLE, pagerank DOUBLE) WITH primary_id_as_attribute="true";
      ADD DIRECTED EDGE sent_eth (from Wallet, to Wallet, amount FLOAT, n_transactions INT, timestamp INT) WITH REVERSE_EDGE="reverse_sent_eth";
  }
  RUN SCHEMA_CHANGE JOB build_schema
''')
print(results)

Using graph 'Ethereum2'
Successfully created schema change jobs: [build_schema2].
Kick off schema change job build_schema2
Doing schema change on graph 'Ethereum2' (current version: 2)
Trying to add local vertex 'Wallet' to the graph 'Ethereum2'.
Trying to add local edge 'sent_eth' and its reverse edge 'reverse_sent_eth' to the graph 'Ethereum2'.

Graph Ethereum2 updated to new version 3
The job build_schema2 completes in 1.075 seconds!


### Create Loading Jobs
We will now create custom loading jobs to map the values from our datasets to vertex and edge attributes for our transaction graph.

#### Wallets

In [36]:
# Custom loading job that maps the values of nodes.csv to VERTEX attributes
results = conn.gsql('''
  USE GRAPH Ethereum
  BEGIN
  CREATE LOADING JOB load_wallets FOR GRAPH Ethereum {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO VERTEX Wallet VALUES($1, $2, $3, $4, $5, _, _, _, _, _, _, _) USING SEPARATOR=",", HEADER="true", EOL="\n";
  }
  END
  ''')
print(results)

Using graph 'Ethereum2'
Successfully created loading jobs: [load_wallets].


#### Transactions

In [37]:
# Custom loading job that maps the values of edges.csv to EDGE attributes
results = conn.gsql('''
  USE GRAPH Ethereum
  BEGIN
  CREATE LOADING JOB load_transactions FOR GRAPH Ethereum {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO EDGE sent_eth VALUES($1, $2, $3, $4, $5) USING SEPARATOR=",", HEADER="true", EOL="\n";
  }
  END''')
print(results)

Using graph 'Ethereum2'
Successfully created loading jobs: [load_transactions3].


### Load Data

Using the loading jobs we just created, we will upload wallet (node) and transaction (edge) data from our local system into the graph stored in TigerGraph.

In [38]:
os.chdir('../data/')

# Load the nodes file with the 'load_wallets' job
nodes_file = 'nodes_train_test_split.csv'
results = conn.runLoadingJobWithFile(filePath=nodes_file, fileTag='MyDataSource', jobName='load_wallets')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 86623,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Wallet",
          "validObject": 86622,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 1,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [40]:
# Load the edges file with the 'load_transactions' job
edges_file = 'edges_engineered.csv'
results = conn.runLoadingJobWithFile(filePath=edges_file, fileTag='MyDataSource', jobName='load_transactions')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 106084,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [],
      "edge": [
        {
          "typeName": "sent_eth",
          "validObject": 106083,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 1,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


### Exploring the Graph
We will now conduct some basic EDA on the Ethereum transaction network. We will also add relevant node features using [GSQL](https://github.com/tigergraph/gsql-graph-algorithms): TigerGraph's graph query langauge and in-database graph data science algorithm library.

In [42]:
# Get Node/Edge Counts
print("Vertex Counts")
for vertex in conn.getVertexTypes():
  print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")

print("--------------")
print("Edge Counts")
for edge in conn.getEdgeTypes():
  print(f"There are {conn.getEdgeCount(edge)} {edge} edges in the graph")

Vertex Counts
There are 86622 Wallet vertices in the graph
--------------
Edge Counts
There are 106083 sent_eth edges in the graph


In [66]:
# Install and run GSQL query for indegree/outdegree
os.chdir('../gsql/')
f = conn.gds.featurizer()
f.installAlgorithm("get_degrees", query_path="get_degrees.gsql")
f.runAlgorithm("get_degrees", custom_query=True)

Installing and optimizing the queries, it might take a minute...
Queries installed successfully
Default parameters are: None


[{'"Success !"': 'Success !'}]

In [67]:
# Install and run GSQL query for total/minimum ETH sent/received
f = conn.gds.featurizer()
f.installAlgorithm("summarize_amounts", query_path="summarize_amounts.gsql")
f.runAlgorithm("summarize_amounts", custom_query=True)

Default parameters are: None


[{'"Success!"': 'Success!'}]

In [69]:
# Install and run GSQL query for pagerank
feat = conn.gds.featurizer()
feat.installAlgorithm("tg_pagerank")
tg_pagerank_params = {
  "v_type": "Wallet",
  "e_type": "sent_eth",
  "result_attribute": "pagerank",
  "top_k":5  
}
results = feat.runAlgorithm("tg_pagerank",tg_pagerank_params)[0]['@@top_scores_heap']
results

[{'Vertex_ID': '30246', 'score': 158.67383},
 {'Vertex_ID': '29111', 'score': 84.32715},
 {'Vertex_ID': '759', 'score': 72.71993},
 {'Vertex_ID': '26668', 'score': 70.40115},
 {'Vertex_ID': '14687', 'score': 65.86597}]